In [77]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import numpy as np

In [78]:
webdriver_service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=webdriver_service)

# URL of the webpage you want to access
url = "https://www.wunderground.com/history/monthly/us/tn/nashville/KBNA/date/2023-6"
driver.get(url)


element = WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.CLASS_NAME, "days"))
)




table= driver.find_element(By.CLASS_NAME,'days')
titles = []

titles_row = table.find_element(By.TAG_NAME, "tr")
titles_items = titles_row.find_elements(By.TAG_NAME, 'td')
for title in titles_items:
    titles.append(title.text)
print(titles)

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.30M/6.30M [00:02<00:00, 2.96MB/s]


['Time', 'Temperature (°F)', 'Dew Point (°F)', 'Humidity (%)', 'Wind Speed (mph)', 'Pressure (in)', 'Precipitation (in)']


In [79]:
# creates a df from each mini table
body = table.find_element(By.TAG_NAME, 'tbody')
inner_t = body.find_elements(By.TAG_NAME, 'table')
inner_tables = []
for inner in inner_t:

    rows_item = inner.find_elements(By.TAG_NAME, 'tr')
    rows = []
    for row in rows_item:
        data_item = row.find_elements(By.TAG_NAME, 'td')
        data = []
        for i in data_item:
            data.append(i.text)
        rows.append(data)
    
    df = pd.DataFrame(rows)
    new_header = df.iloc[0] 
    df = df[1:] 
    df.columns = new_header 

    inner_tables.append(df)


In [80]:
# build final df

cols = []
data = []
for i in range(len(titles)):
    for j in range(len(inner_tables[i].columns)):
        cols.append(inner_tables[i].columns[j] + ' ' + titles[i])
        data.append(inner_tables[i][inner_tables[i].columns[j]])

cols[0] = "Time"
df = pd.DataFrame(data = np.array(data)).transpose()
df.columns = cols
df

,Time,Max Temperature (°F),Avg Temperature (°F),Min Temperature (°F),Max Dew Point (°F),Avg Dew Point (°F),Min Dew Point (°F),Max Humidity (%),Avg Humidity (%),Min Humidity (%),Max Wind Speed (mph),Avg Wind Speed (mph),Min Wind Speed (mph),Max Pressure (in),Avg Pressure (in),Min Pressure (in),Total Precipitation (in)
0,1,87,76.1,67,67,64.0,62,90,67.7,43,10,3.2,0,29.4,29.4,29.3,0.00
1,2,91,79.7,67,67,61.8,56,90,58.1,33,12,6.1,0,29.4,29.3,29.2,0.00
2,3,91,79.2,64,60,56.5,51,80,49.2,25,14,5.4,0,29.3,29.2,29.2,0.00
3,4,91,78.3,65,66,61.2,58,87,58.3,34,15,4.1,0,29.3,29.2,29.2,0.00
4,5,91,78.7,66,65,61.6,56,90,58.8,37,12,4.8,0,29.3,29.3,29.3,0.10
5,6,89,78.0,64,60,54.7,48,84,48.2,26,10,3.6,0,29.3,29.3,29.2,0.00
6,7,82,72.3,61,63,57.8,54,83,62.1,38,10,5.1,0,29.2,29.2,29.1,0.00
7,8,82,72.3,64,62,53.5,41,90,57.5,24,12,5.0,0,29.2,29.2,29.1,0.00
8,9,83,71.5,56,54,48.1,40,84,48.2,22,9,2.6,0,29.3,29.3,29.2,0.00
9,10,89,74.5,57,61,55.5,51,81,54.4,33,10,4.3,0,29.3,29.2,29.1,0.00
